### Functions 

In [2]:
import rasterio 
import gdal
import numpy as np
import pandas as pd

def CreateRaster(lon,lat,data,resx,resy,save_directory):
    from osgeo import gdal
    from osgeo import gdal_array
    from osgeo import osr,ogr

    xmin,ymin,xmax,ymax = [lon.min(),lat.min(),lon.max(),lat.max()]
    nrows,ncols = np.shape(data)
    geotransform=(xmin-resy,resy,0,ymax+resx,0, -resx)  

    output_raster = gdal.GetDriverByName('GTiff').Create(save_directory,ncols, nrows, 1 ,gdal.GDT_Float32)  # Open the file

    output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
    srs = osr.SpatialReference()                 # Establish its coordinate encoding
    srs.ImportFromEPSG(4326)                     # This one specifies WGS84 lat long.

    output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system                                         
    output_raster.GetRasterBand(1).WriteArray(data)   # Writes my array to the raster
    output_raster.FlushCache()
    return

#calculation for Hamonds evaporation
def evap(T,date,lat,long,K):
    "Hamonds PET calculations"
    Vs=6.108*np.exp(17.26939*T/(T+273.3))
    Vd= 216.7*(Vs/(273.3+T))
    
    #sunrise and sunset
    sun = Sun(lat, long)
    sr = sun.get_local_sunrise_time(date)
    ss = sun.get_local_sunset_time(date)

    #hours of sun
    hr=(sr-ss).total_seconds()/3600
    
    D= hr/12.
    PET=0.1651*D*Vd*K
    
    return PET

### Calculate monthly PET Arrays

In [4]:
from suntime import Sun, SunTimeException
import datetime
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
for year in np.arange(1981,2020):
    print(year)
    #import temperature data
    T= pd.read_pickle(path+'Temp/Daily_Vars/t2m_%s.pkl'%year)
    
    #import lat and long
    lat= np.load(path+'Temp/lat.npy')
    long= np.load(path+'Temp/long.npy')

    for month in np.arange(1,13,1):
        Day=[]
        for day in np.arange (1,32,1):
            #extract daily temperatures
            try:
                T_W=T[(T.Month==month)&(T.Day==day)].Var.values[0]
            except IndexError:
                continue
            Date=datetime.date(year, month, day)
            
            
            Shape=np.shape(T_W)
            
            #create raster for Hamond PET
            K=1.4
            PE=np.empty([Shape[0],Shape[1]])
            for i in np.arange(0,Shape[0]):
                for j in np.arange(0,Shape[1]):
                    
                    #calculate PET for each grid cell
                    TEMP= T_W[i,j]
                    LT= lat[i]
                    LG=long[i]
                    E0= evap(TEMP,Date,LT,LG,K)
                    
                    PE[i][j]=E0
                
            Day.append(PE)
            
        #sum daily PET to monthly PET
        M=np.nansum(np.array(Day),axis=0)  
        
        #save monthly PET rasters
        np.save(path+'Potential_Evap/Calc_Monthly/Ham_Max/%s_%s_Ham'%(year,month),M)

2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


ValueError: day is out of range for month

### Convert monthly arrays to rasters 

In [5]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
Resx= 0.1 #x resolution
Resy= 0.1 #y resolution
resample_value=6 # how much to decrease resolution by in resampling

#lat and long
lat=np.load(path+'Potential_Evap/Lat_PET.npy')
long=np.load(path+'Potential_Evap/Long_PET.npy')

for year in np.arange(1981,2019,1):
    for month in np.arange(1,13,1):
        #import monthly array
        x= np.load(path+'Potential_Evap/Calc_Monthly/Ham_Max/%s_%s_Ham.npy'%(year,month))

        #create array with data and lat,long
        hold=[]
        Z_ERA5=[]
        for i in np.arange(0,len(lat)):
            for j in np.arange(0,len(long)):
                Z_ERA5.append([x[i][j],float(lat[i]),float(long[j])])
        hold.append(Z_ERA5) 
        
        
        #extract raster array to work with
        working=(np.array(hold)[0])

        #used for reshaping arrays
        shp=len(np.unique(np.array(working)[:,2]))

        #where to save the raster to
        Save_file=path+'Potential_Evap/Raster_Calc_PET/MAKE.tif'

        # reshape data dimensions
        precip_array_ERA5=np.reshape(working[:,0],(-1,shp))
        lat_array_ERA5=np.reshape(working[:,1], (-1,shp))
        lon_array_ERA5=np.reshape(working[:,2],(-1,shp))

        # Create the raster
        CreateRaster(lon=lon_array_ERA5,lat=lat_array_ERA5,data=precip_array_ERA5, resx=Resx, resy=Resy, save_directory=Save_file)

        # where to save resmapled raster to
        Save_file_N=path+'Potential_Evap/Raster_Calc_PET/%s_%s_Max_Ham.tif'%(year,month) 

        # resample raster 
        with rasterio.open(Save_file) as dataset:

            dataP = dataset.read(1, out_shape=(dataset.height * resample_value, dataset.width * resample_value))

            res_new = 0.1/resample_value
            lat_array_new_P=np.arange(lat_array_ERA5.max()+(res_new*(resample_value-1)),lat_array_ERA5.min()-0.001,-res_new)
            lat_array_new_P=np.array([lat_array_new_P]*(np.shape(dataP)[1])).transpose()

            lon_array_new_P=np.arange(lon_array_ERA5.max(),lon_array_ERA5.min()-(res_new*(resample_value-1))-0.001, -res_new)
            lon_array_new_P=np.array([lon_array_new_P]*(np.shape(dataP)[0]))

        #create resampled raster
        CreateRaster(lon_array_new_P,lat_array_new_P,dataP,res_new, res_new, Save_file_N)
        
        #delete non-reampled raster
        gdal.GetDriverByName('GTiff').Delete(Save_file)


        # save the new lat and long coordinates for resampled rasters
        np.save(path+'Lat_Ham_Max',np.array(lat_array_new_P))
        np.save(path+'Long_Ham_Max',np.array(lon_array_new_P))

### Creating average watershed PET

In [6]:
import  fiona
import rasterio.mask

#watersheds
Watersheds=['Spius','Chilko','Chilcotin','Nation','Osilinka','Mesilinka','Stellako','Nautley']

path='D:/UNBC/HydroMet_Project/Data/ERA5_land/Potential_Evap/'

for WS in Watersheds:

    Res=pd.DataFrame()
    for year in np.arange(1981,2019):
        for month in np.arange(1,13):

            #extract shapefile of watershed
            with fiona.open("D:/UNBC/HydroMet_Project/GIS_maps/Shapefiles_WaterFeatures/Individual_ws/%s/%s_WS.shp"%(WS,WS), "r") as shapefile:
                features = [feature["geometry"] for feature in shapefile]

            #clip rasterr to watershed
            with rasterio.open(path+'Raster_Calc_PET/%s_%s_Max_Ham.tif'%(year,month) ) as src:
                out_image, out_transform = rasterio.mask.mask(src, features,crop=True,nodata=np.nan)
                out_meta = src.meta.copy()
            
            #find the average monthly PET for watershed and document it
            Res=Res.append({'Year':year,'Month':month,'PE':np.nanmean(out_image)},ignore_index=True)
    
    #save results to excel
    Res.to_excel(path+'%s_Ham_Max.xlsx'%WS)